In [1]:
from google.colab import drive

# Mount your Drive at /content/drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q -U transformers accelerate bitsandbytes langchain-community chromadb sentence-transformers
# ───────────────────────────────────────────────────────────────────────────────
# • `transformers`          — Hugging Face’s library of pre-trained NLP models and tokenizers
# • `accelerate`            — Utility for easy device placement and distributed inference/training
# • `bitsandbytes`          — Provides 4-bit and 8-bit quantization routines to reduce model memory footprint
# • `langchain-community`   — Community-contributed extensions for LangChain, including extra embedders, tools, and integrations
# • `chromadb`              — Open source, high-performance vector database for storing and retrieving embeddings

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 138.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 118.6 MB/s e

In [3]:
import torch  # PyTorch core library for tensor operations and hardware acceleration
import time   # Standard library for measuring elapsed time during inference or other operations

In [4]:
import os                                   # Provides functions to interact with the operating system (e.g., environment variables)
from huggingface_hub import login           # Function to authenticate with Hugging Face Hub programmatically
from getpass import getpass                 # Securely prompt the user for sensitive input without echoing it on-screen

# 1) Prompt the user to enter their Hugging Face access token.
#    getpass() ensures the token is not displayed as they type.
hf_token = getpass("🔐 Enter your Hugging Face token: ")

# 2) Authenticate to Hugging Face using the provided token.
#    add_to_git_credential=False prevents storing the token in your git config.
login(token=hf_token, add_to_git_credential=False)

# 3) Store the token in an environment variable.
#    Many Hugging Face utilities will look for HF_TOKEN if no token is passed explicitly.
os.environ["HF_TOKEN"] = hf_token

🔐 Enter your Hugging Face token: ··········


In [8]:
# Module 1: Load Libraries & Initialize Models for RAG Pipeline (with correct embedder max length)

from sentence_transformers import SentenceTransformer
#from langchain_community.embeddings import SentenceTransformerEmbeddings
#from langchain_community.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.text_splitter import TokenTextSplitter
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)

# 1. Quantization config for Mistral 7B
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16"
)

RAG_MODEL_ID = "mistralai/Mistral-7B-v0.1"

# 2. Load tokenizer & quantized model for RAG generation
tokenizer = AutoTokenizer.from_pretrained(RAG_MODEL_ID, token=True)
model     = AutoModelForCausalLM.from_pretrained(
    RAG_MODEL_ID,
    quantization_config=quant_config,
    device_map="auto",
    token=True
)

# 3. Wrap in a Hugging Face text-generation pipeline (limit new tokens)
hf_pipeline = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=200
)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# 4. Initialize the Sentence‑Transformer embedder
EMBED_MODEL_ID = "all-MiniLM-L6-v2"
embeddings = SentenceTransformerEmbeddings(
    model_name=EMBED_MODEL_ID,
    model_kwargs={"device": "cuda"}
)

# 5. Determine the embedder’s true max token window via sentence-transformers
st = SentenceTransformer(EMBED_MODEL_ID)
embedder_max = st.max_seq_length  # typically 256

# 6. Configure a token‑based splitter safely under that limit (e.g., 200 tokens)
chunker = TokenTextSplitter(
    encoding_name="cl100k_base",
    chunk_size=200,   # safely below embedder_max
    chunk_overlap=50  # context overlap
)
# 7. Initialize ChromaDB for vector storage & retrieval
#chroma = Chroma(
  #  embedding_function=embeddings
#)

# 7. Mount Google Drive and persist Chroma there
#from google.colab import drive
#drive.mount('/content/drive')

#persist_path = "/content/drive/MyDrive/GenAI/chroma_db"

#chroma = Chroma(
 #   persist_directory=persist_path,
  #  embedding_function=embeddings
#)

#retriever = chroma.as_retriever()




Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [9]:
!pip install -U pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 21.2 MB/s eta 0:00:00


In [10]:
# Module 2+: Load PDFs, Aggregate Text, and Count Characters

import os                                       # For directory listing
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document            # Standard LangChain Document

# 1. Directory containing PDFs
pdf_dir = "/content/drive/MyDrive/GenAI/Assignment_2"

# 2. Prepare storage
documents = []     # To hold Document objects
all_text = ""      # To aggregate all page contents

# 3. Loop over each file in the directory
#REMOVE TABLE OF CONTENTS

for fname in os.listdir(pdf_dir):
    if fname.lower().endswith(".pdf"):
        path = os.path.join(pdf_dir, fname)
        loader = PyPDFLoader(path)
        docs = loader.load()
        for i, doc in enumerate(docs):
            if i <= 5:  # Skip first 6 pages
                continue
            doc.metadata["source"] = fname
            documents.append(doc)
            all_text += doc.page_content


#for fname in os.listdir(pdf_dir):
 #   if fname.lower().endswith(".pdf"):         # Only PDFs
  #      path = os.path.join(pdf_dir, fname)
   #     loader = PyPDFLoader(path)             # Initialize loader
    #    docs = loader.load()                   # Extract pages as Document objects
     #   for doc in docs:
            #doc.metadata["source"] = fname     # Tag each page with filename
            #documents.append(doc)
            #all_text += doc.page_content       # Append page text to aggregate

# 4. Compute total number of characters
total_chars = len(all_text)
print(f"Total characters across all documents: {total_chars}")


from langchain_community.vectorstores import Chroma

persist_path = "/content/drive/MyDrive/GenAI/chroma_db"

# Create and save the vector store
chroma = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=persist_path
)

#chroma.persist()
chroma = Chroma(
    persist_directory=persist_path,
    embedding_function=embeddings
)
retriever = chroma.as_retriever()

Total characters across all documents: 112297


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/tmp/ipython-input-1952590552.py:56: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  chroma.persist()


In [ ]:
documents

In [13]:
import re

def simple_clean(text: str) -> str:
    """
    Clean raw PDF-to-text artifacts:
      1. Remove form feeds (page breaks)
      2. Fix hyphenation at line ends
      3. Collapse single newlines into spaces (but keep double newlines)
      4. Remove non-printable/control chars
      5. Normalize multiple spaces & newlines
    """
    # 1. Remove PDF page breaks
    text = text.replace('\f', ' ')

    # 2. Join hyphenated line‑break words: "exam-\nple" → "example"
    text = re.sub(r'-\s*\n\s*', '', text)

    # 3. Normalize Windows/Mac line endings to \n
    text = text.replace('\r\n', '\n').replace('\r', '\n')

    # 4. Collapse single newlines into spaces (but preserve paragraph breaks)
    #    A single newline preceded/followed by text becomes a space;
    #    two or more newlines stay as double‑newline.
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)

    # 5. Remove any remaining non‑printable or weird Unicode
    text = re.sub(r'[^\x20-\x7E\n]', '', text)

    # 6. Collapse runs of spaces to one
    text = re.sub(r' {2,}', ' ', text)

    # 7. Collapse runs of 3+ newlines down to exactly two (paragraph break)
    text = re.sub(r'\n{3,}', '\n\n', text)

    return text.strip()

# Apply to each Document and count sizes:
cleaned_texts = []
total_before = 0
total_after  = 0

for doc in documents:
    raw = doc.page_content
    total_before += len(raw)
    cleaned = simple_clean(raw)
    total_after  += len(cleaned)
    cleaned_texts.append(cleaned)

print(f"Total chars before cleaning: {total_before:,}")
print(f"Total chars after  cleaning: {total_after:,}")



Total chars before cleaning: 112,297
Total chars after  cleaning: 109,576


In [14]:
cleaned_texts

['7 Standard Deduction (2026) Filing Status Current Law Provision Single $8,300 $16,300 Head of Household $12,150 $24,500 Married Filing Jointly $16,600 $32,600 ** Joint Committee of Taxation, 2026 Projection Sec. 110003. Termination of deduction for personal exemptions. Current Law: Under current law, the deduction for personal exemptions is set to return after December 31, 2025. Provision: This provision permanently repeals the deduction for personal exemptions. Sec. 110004. Extension of increased child tax credit and temporary enhancement. Current Law: Under current law, the child tax credit will return to pre-2017 levels after December 31, 2025. This means that the credit amount will drop from $2,000 to $1,000 per child, the child Social Security number (SSN) requirement will be eliminated, and fewer American families will qualify for the credit as the income phase-out levels return to much lower thresholds. Additionally, the $500 nonrefundable credit for non-child dependents will 

In [15]:
# Module 5: Chunk Cleaned Text with Token‑Level Splitter & Embed into ChromaDB

from langchain.text_splitter import TokenTextSplitter
from langchain.schema import Document

# 1. Initialize a token‑based splitter with fixed size and overlap
splitter = TokenTextSplitter(
    encoding_name="cl100k_base",  # Must match your tokenizer’s BPE encoding
    chunk_size=200,               # Up to 200 tokens per chunk
    chunk_overlap=50              # 50 tokens overlap for contextual continuity
)

# 2. Collect all chunked Document objects
chunked_docs = []

# 3. Iterate over each original document and its cleaned text
#    `documents` holds metadata; `cleaned_texts` holds the cleaned content
for original_doc, cleaned_text in zip(documents, cleaned_texts):

    # 3a. Split the cleaned text into token‑sized chunks
    splits = splitter.split_text(cleaned_text)

    # 3b. Wrap each chunk in a LangChain Document, preserving metadata
    for idx, chunk_text in enumerate(splits):
        meta = original_doc.metadata.copy()
        meta["chunk"] = idx
        doc_chunk = Document(page_content=chunk_text, metadata=meta)
        chunked_docs.append(doc_chunk)

# 4. Extract texts and metadata for bulk embedding
texts     = [doc.page_content for doc in chunked_docs]
metadatas = [doc.metadata     for doc in chunked_docs]

# 5. Embed and persist all chunks into ChromaDB
#    Generates vector embeddings for each chunk and saves them with metadata
chroma.add_texts(texts=texts, metadatas=metadatas)



['0e974fc1-cfd5-49b1-b42a-45b2a666e40d',
 'a157238a-bbe1-40c2-8311-9335ca164107',
 '883a0f47-45da-4098-9e89-7af70b5d4f60',
 '046e009d-eede-49d3-a772-7b3d536042a6',
 '0b458400-90bb-429c-8689-1eb7ec0c138f',
 '78b8c380-b6af-474b-8a8b-704a7bf88dca',
 'd9b96fe5-7dfe-4e74-9339-2875a1d0f02c',
 '124e3a5f-6af3-4e49-bf6c-8bb08a8b3f13',
 'fd6d4a2f-e4d1-44a3-b4ea-92c20b42fc1a',
 '82372bf1-c185-4914-8700-aeb42d5258b3',
 '004774a5-40b8-4874-8ce9-430a5d133ea1',
 'd47ec349-e08c-4cf8-8516-965374b31b05',
 'd012e489-f161-434f-9c26-c44663d1c6fc',
 'd52fef6a-0565-4c64-873a-08d505a6dc74',
 '8d218d1e-5c93-4c98-80fb-597b67faffd5',
 '77e32f7c-b9f4-488c-ac99-338622d543f5',
 'd88de403-861d-40b2-a4cf-676679363ab3',
 '02e08e43-2567-4e55-a0b4-2e84d79491ef',
 '2f99ad2c-25e0-4d49-86a3-76c405fefbf9',
 '66d0e288-7dbe-44b8-a916-de24bc88c121',
 '88d9362f-b808-4d39-8dde-96635dbc02c5',
 'd3d3712d-0cc9-4a62-bf79-3aa93b4d51eb',
 '4d54d4ab-3886-4a0b-afb4-2c2aa6b845a7',
 '0c956135-d36d-4c4a-a2da-a6370e5c1007',
 'e4249a3d-4fb8-

In [16]:
# ─── Interactive RAG Chain with Tuned Hyperparameters ───

# 1. Imports
from transformers import pipeline, AutoTokenizer
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# 2. Reconfigure your text-generation pipeline
#    • allow up to 500 tokens in the model’s answer
#    • note: `tokenizer` and `model` are from your RAG setup
hf_pipeline = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=500        # ← limit answer to 500 tokens
)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# 3. Define your zero-shot prompt template
prompt_template = """
You are a helpful policy analyst AI. Use the following context from the Big Beautiful Bill to answer the question as clearly and accurately as possible. Be thorough, cite specific sections or clauses where relevant, and avoid speculation.
Context:
{context}

Question: {question}
Answer:
"""
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template
)

# 4. Compute static prompt token usage (for context-window planning)
#    • Mistral 7B window ≈ 8000 tokens; reserve ~64 as buffer
#    • this helps you know how much context you can safely pass
static_example = prompt_template.format(context="", question="")
static_tokens = len(tokenizer(static_example).input_ids)
print(f"Static prompt uses {static_tokens} tokens")
effective_context_window = 8000 - static_tokens - 500 - 64
print(f"Max context tokens available: {effective_context_window}")

# 5. Build the retriever, retrieving more chunks (k=10)
retriever = chroma.as_retriever(search_kwargs={"k": 10})

# 6. Build the RetrievalQA chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",               # concatenate all retrieved context
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

# 7. Prepare a tokenizer for user-question length checking (limit 300 tokens)
qtok = AutoTokenizer.from_pretrained(RAG_MODEL_ID, use_auth_token=True)

# 8. Interactive loop for user questions
while True:
    user_q = input("\nEnter your question (blank to quit): ").strip()
    if not user_q:
        print("Goodbye!")
        break

    # 8a. Enforce user question ≤300 tokens
    qlen = len(qtok(user_q).input_ids)
    if qlen > 300:
        print(f"[Error] Question is {qlen} tokens long; limit to 300 tokens.")
        continue

    # 8b. Run the RAG chain: retrieve context & generate answer
    result = rag_chain(user_q)

    # 8c. Display the generated answer
    print("\n=== Answer (max 500 tokens) ===")
    print(result["result"].strip())

    # 8d. Show which source chunks were used
    print("\n=== Source Chunks (k=10) ===")
    for doc in result["source_documents"]:
        src      = doc.metadata.get("source", "unknown")
        chunk_id = doc.metadata.get("chunk", "?")
        print(f"• {src} (chunk {chunk_id})")



Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:999: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Static prompt uses 63 tokens
Max context tokens available: 7373

Enter your question (blank to quit): 
Goodbye!


In [18]:
# ─── Add LLM‑as‑Judge with Detailed Rubric & Proper Scoring ───

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline as hf_pipeline_fn
from langchain.llms import HuggingFacePipeline
from langchain import LLMChain
from langchain.prompts import PromptTemplate

# 1. Load & wrap judge model (Phi 1.5B)
JUDGE_MODEL_ID = "microsoft/phi-1_5"
judge_tokenizer = AutoTokenizer.from_pretrained(JUDGE_MODEL_ID, use_auth_token=True)
judge_model     = AutoModelForCausalLM.from_pretrained(
    JUDGE_MODEL_ID, trust_remote_code=True, device_map="auto", use_auth_token=True
)
judge_pipeline = hf_pipeline_fn(
    task="text-generation",
    model=judge_model,
    tokenizer=judge_tokenizer,
    return_full_text=False,
    max_new_tokens=200
)
judge_llm = HuggingFacePipeline(pipeline=judge_pipeline)

# 2. Define rubric text
rubric_text = """
Rubric for evaluation:
Faithfulness:
  1: Major hallucinations or contradictions.
  2: Significant inaccuracies; some correct.
  3: Partially faithful; most key points correct.
  4: Mostly faithful; minor omissions.
  5: Fully faithful and accurate.

Relevance:
  1: Unrelated to question/context.
  2: Minimally related.
  3: Somewhat relevant; misses aspects.
  4: Mostly relevant; minor gaps.
  5: Highly relevant; fully addresses question.
"""

# 3. Build judge prompt including rubric
judge_prompt = PromptTemplate(
    input_variables=["question", "answer", "context"],
    template=rubric_text + """
Question:
{question}

Answer:
{answer}

Context:
{context}

Provide:
1) Faithfulness (1–5):
2) Relevance    (1–5):
Justification:
"""
)
judge_chain = LLMChain(llm=judge_llm, prompt=judge_prompt)

# 4. Interactive RAG + Judge loop (assumes rag_chain & qtok defined)
while True:
    user_q = input("\nEnter question (blank to quit): ").strip()
    if not user_q:
        break

    # Enforce ≤300 tokens
    if len(qtok(user_q).input_ids) > 300:
        print("[Error] Question too long.")
        continue

    # Run RAG chain
    rag_res = rag_chain(user_q)
    answer  = rag_res["result"].strip()
    docs    = rag_res["source_documents"]

    print("\n=== RAG Answer ===\n", answer)

    # Prepare context
    ctx = "\n\n".join(d.page_content for d in docs[:3])

    # Display rubric once
    print(rubric_text)

    # Run judge evaluation
    eval_out = judge_chain.run({
        "question": user_q,
        "answer":   answer,
        "context":  ctx
    })

    print("\n=== Judge Evaluation ===\n", eval_out)
    print("\n" + "="*60)


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:999: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Device set to use cuda:0



Enter question (blank to quit): How is medicare affected by this bill?


/tmp/ipython-input-3954121146.py:74: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  rag_res = rag_chain(user_q)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



=== RAG Answer ===
 Question: What are the provisions of the bill?
Answer:

Question: What is the context of the Big Beautiful Bill?
Answer: The Big Beautiful Bill is a proposal to reform the Medicare program, which is a federal program that provides health insurance to Americans over the age of 65. The bill includes provisions to reduce the cost of Medicare, improve the quality of care, and expand access to care for all Americans.

Question: What is the context of the Big Beautiful Bill?
Answer: The Big Beautiful Bill is a proposal to reform the Medicare program, which is a federal program that provides health insurance to Americans over the age of 65. The bill includes provisions to reduce the cost of Medicare, improve the quality of care, and expand access to care for all Americans.

Question: What is the context of the Big Beautiful Bill?
Answer: The Big Beautiful Bill is a proposal to reform the Medicare program, which is a federal program that provides health insurance to Americ

/tmp/ipython-input-3954121146.py:87: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  eval_out = judge_chain.run({



=== Judge Evaluation ===
 1) Faithfulness (1–5):
   a. Major hallucinations or contradictions.
   b. Significant inaccuracies; some correct.
   c. Partially faithful; some key points correct.
   d. Mostly faithful; minor omissions.
   e. Fully faithful and accurate.
2) Relevance (1–5):
   a. Unrelated to question/context.
   b. Minimally related.
   c. Somewhat relevant; minor gaps.
   d. Mostly relevant; minor gaps.
   e. Highly relevant; fully addresses question.



Title: The Marvels of Modern Technology: A Contrastive Analysis

Introduction:

Modern technology has revolutionized our lives in countless ways, offering an array of benefits, but also presenting some challenges. In this essay, we will explore the wonders of modern technology, focusing on sensors and inputs, while using the contrasting tones of brilliance and dullness


Enter question (blank to quit): 


In [12]:
!zip -r /content/chroma_db.zip /content/drive/MyDrive/GenAI/chroma_db


  adding: content/drive/MyDrive/GenAI/chroma_db/ (stored 0%)
  adding: content/drive/MyDrive/GenAI/chroma_db/chroma.sqlite3 (deflated 65%)
  adding: content/drive/MyDrive/GenAI/chroma_db/53d6180b-d495-45ff-a3bd-47ef21f49e42/ (stored 0%)
  adding: content/drive/MyDrive/GenAI/chroma_db/53d6180b-d495-45ff-a3bd-47ef21f49e42/header.bin (deflated 61%)
  adding: content/drive/MyDrive/GenAI/chroma_db/53d6180b-d495-45ff-a3bd-47ef21f49e42/data_level0.bin (deflated 100%)
  adding: content/drive/MyDrive/GenAI/chroma_db/53d6180b-d495-45ff-a3bd-47ef21f49e42/length.bin (deflated 84%)
  adding: content/drive/MyDrive/GenAI/chroma_db/53d6180b-d495-45ff-a3bd-47ef21f49e42/link_lists.bin (stored 0%)
